### Lesson 5: Retrieval

Retrieval is the centerpiece of our retrieval augmented generation (RAG) flow.

#### Vectorstore retrieval

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

#### Similarity Search

In [2]:
from langchain_community.embeddings.cohere import CohereEmbeddings
from langchain_community.vectorstores.chroma import Chroma

In [3]:
persist_directory = "./.chroma/"

In [4]:
embedding = CohereEmbeddings()

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding,
)

In [5]:
vectordb._collection.count()

208

In [6]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [7]:
# in-memory
smalldb = Chroma.from_texts(texts=texts, embedding=embedding)

In [8]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [9]:
smalldb.similarity_search(query=question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

In [10]:
smalldb.max_marginal_relevance_search(query=question, k=2, fetch_k=3)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.')]

#### Addressing Diversity: Maximum marginal relevance

In last lesson we introduced one problem: how to enforce diversity in the search results.

**Maximum marginal relevance** strives to achieve both relevance to the query and diversity among the results.

In [11]:
question = "What did they say about matlab?"

doc_ss = vectordb.similarity_search(query=question, k=3)

In [12]:
print(doc_ss[0].page_content[:100])

those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 


In [13]:
print(doc_ss[1].page_content[:100])

those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 


Now with MMR:

In [14]:
docs_mmr = vectordb.max_marginal_relevance_search(query=question, k=3)

In [15]:
print(docs_mmr[0].page_content[:100])

those homeworks will be done in either MATLA B or in Octave, which is sort of — I 
know some people 


In [16]:
print(docs_mmr[1].page_content[:100])

into his office and he said, "Oh, professo r, professor, thank you so much for your 
machine learnin


#### Addressing Specificity: working with metadata

In last lesson, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on **metadata**.

**metadata** provides context for each embedded chunk.

In [17]:
question = "what did they say about regression in the third lecture?"

In [18]:
docs = vectordb.similarity_search(
    query=question,
    k=3,
    filter={"source": "docs/cs229_lectures/MachineLearning-Lecture03.pdf"},
)

In [19]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 6, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}


### Addressing Specificity: working with metadata using self-query retriever

But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use `SelfQueryRetriever`, which uses an LLM to extract:

1. The query string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [20]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_community.llms.cohere import Cohere

In [21]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The lecture the chunk is from, \
            should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, \
            `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, \
            or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the lecture",
        type="integer",
    ),
]

In [22]:
document_content_description = "Lecture notes"

llm = Cohere()

retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectordb,
    document_contents=document_content_description,
    metadata_field_info=metadata_field_info,
    verbose=True,
)

In [23]:
question = "what did they say about regression in the third lecture?"

In [24]:
docs = retriever.get_relevant_documents(query=question)

In [25]:
for d in docs:
    print(d.metadata)

{'page': 2, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 2, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}
{'page': 0, 'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf'}


#### Additional tricks: compression

Another approach for improving the quality of retrieved docs is compression.

Information most relevant to a query may be buried in a document with a lot of irrelevant text.

Passing that full document through your application can lead to more expensive LLM calls and poorer responses.

Contextual compression is meant to fix this.

In [26]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [27]:
def pretty_print_docs(docs: list) -> None:
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [28]:
llm = Cohere()

compressor = LLMChainExtractor.from_llm(llm=llm)

In [29]:
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(),
)

In [30]:
question = "what did they say about matlab?"

compressed_docs = compressor_retriever.get_relevant_documents(query=question)

/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead p

In [31]:
compressed_docs

[Document(page_content=">>>\nthose homeworks will be done in either MATLAB or in Octave, which is sort of — I \nknow some people call it a free version of MATLAB, which it sort  of is, sort of isn't. \nSo I guess for those of you that haven't seen MATLAB before, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to \nplot data. And it's sort of an extremely easy to  learn tool to use for implementing a lot of \nlearning algorithms. \nAnd in case some of you want to work on your  own home computer or something if you \ndon't have a MATLAB license, for the purposes of  this class, there's also — [inaudible] \nwrite that down [inaudible] MATLAB — there' s also a software package called Octave \nthat you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it's free, and for the purposes of  this class, it will work for just about \neveryt

In [32]:
pretty_print_docs(compressed_docs)

Document 1:

>>>
those homeworks will be done in either MATLAB or in Octave, which is sort of — I 
know some people call it a free version of MATLAB, which it sort  of is, sort of isn't. 
So I guess for those of you that haven't seen MATLAB before, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to 
plot data. And it's sort of an extremely easy to  learn tool to use for implementing a lot of 
learning algorithms. 
And in case some of you want to work on your  own home computer or something if you 
don't have a MATLAB license, for the purposes of  this class, there's also — [inaudible] 
write that down [inaudible] MATLAB — there' s also a software package called Octave 
that you can download for free off the Internet. And it has somewhat fewer features than MATLAB, but it's free, and for the purposes of  this class, it will work for just about 
everything. 
>>>

Would you

#### Combining various techniques

In [33]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever(search_type="mmr"),
)

In [34]:
question = "what did they say about matlab?"

compressed_docs = compression_retriever.get_relevant_documents(
    query=question,
)

/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/home/zeyang/git-repos/ai-courses/deeplearning_ai/langchain_chat_with_data/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:321: UserWarning: The predict_and_parse method is deprecated, instead p

In [35]:
pretty_print_docs(compressed_docs)

Document 1:

>>>
those homeworks will be done in either MATLAB or in Octave, which is sort of — I 
know some people call it a free version of MATLAB, which it sort  of is, sort of isn't. 
So I guess for those of you that haven't seen MATLAB before, MATLAB is I guess part of the programming language that makes it very easy to write codes using matrices, to write code for numerical routines, to move data around, to 
plot data. And it's sort of an extremely easy to  learn tool to use for implementing a lot of 
learning algorithms. 
And in case some of you want to work on your  own home computer or something if you 
don't have a MATLAB license, for the purposes of  this class, there's also — [inaudible] 
write that down [inaudible] MATLAB — there' s also a software package called Octave 
that you can download for free off the Internet. 
So actually I, well, so yeah, just a side comment for those of you that haven't seen 
MATLAB before I guess, once a colleague of mine at a different univer

#### Other types of retrieval

It's worth noting that vectordb as not the only kind of tool to retrieve documents.

The `LangChain` retriever abstraction includes other ways to retrieve documents, such as **TF-IDF** or **SVM**.

In [36]:
from langchain.retrievers import SVMRetriever, TFIDFRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [37]:
pages = PyPDFLoader(
    file_path="./docs/cs229_lectures/MachineLearning-Lecture01.pdf"
).load()

all_page_text = [p.page_content for p in pages]

joined_page_text = " ".join(all_page_text)

In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)

splits = text_splitter.split_text(text=joined_page_text)

In [ ]:
svm_retriever = SVMRetriever.from_texts(texts=splits, embeddings=embedding)

tfidf_retriever = TFIDFRetriever.from_texts(texts=splits)

In [ ]:
question = "What are major topics for this class?"

docs_svm = svm_retriever.get_relevant_documents(query=question)

docs_svm[0]

In [ ]:
question = "what did they say about matlab?"

docs_tfidf = tfidf_retriever.get_relevant_documents(query=question)

docs_tfidf[0]